In [8]:
# Importing useful libraries and supressing the warning messages

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append("..")
warnings.filterwarnings('ignore')

In [9]:
# Importing some scripts

from scripts.data_visualization import Data_Viz 
from scripts.data_cleaning import DataCleaner
#from scripts.visualization_logger import logger

DC = DataCleaner()
DV = Data_Viz()

In [10]:
# Importing clean call center data

call_center_clean = pd.read_excel(r'C:\Users\hp\OneDrive - students.jkuat.ac.ke\Desktop\Tiko\call_center_clean.xlsx')
call_center_clean.head()

,Rafiki Id,Call status,Gender,Year of birth,Visited facility,Were you given a tiko card?,Satisfaction,Feedback,Concerns,Notes,Month
0,1920703,Not reachable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,June
1,2016625,Not reachable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,June
2,2013477,Not reachable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,June
3,2016141,Not reachable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,June
4,1970498,Not reachable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,June


In [11]:
# Checking unique encounters on call status

call_center_clean['Call status'].unique()

array(['Not reachable', 'No answer', 'Call later', 'Call successful',
       'Wrong recipient', 'Call rejected'], dtype=object)

In [12]:
call_center_clean['Month'].unique()

array(['June', 'February', 'July', 'May'], dtype=object)

In [13]:
call_center_clean['Notes'].unique()

array([nan,
       "The sister uses his number to get tiko services but right now he's away from her for some time"],
      dtype=object)

In [14]:
del call_center_clean['Rafiki Id']
call_center_clean.head()

,Call status,Gender,Year of birth,Visited facility,Were you given a tiko card?,Satisfaction,Feedback,Concerns,Notes,Month
0,Not reachable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,June
1,Not reachable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,June
2,Not reachable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,June
3,Not reachable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,June
4,Not reachable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,June


In [15]:
# Statistics summary

call_center_clean.describe(include='all').T

,count,unique,top,freq
Call status,1389,6,Not reachable,1123
Gender,141,2,Female,130
Year of birth,140,3,2004-2009,77
Visited facility,140,2,Yes,124
Were you given a tiko card?,124,2,No,101
Satisfaction,124,2,Yes,123
Feedback,124,5,the services were good,57
Concerns,98,6,no,93
Notes,1,1,The sister uses his number to get tiko service...,1
Month,1389,4,June,1385


Inferences from the above statistics
--> Most rafikis were unreachable
--> The female gender topped in seeking the services
--> The age that recorded the highest number are 15-20 year olds
--> Most Rafikis were not issued a Tiko card
--> The satisfaction rate was high
--> Good feedback
--> There were minimal concerns about the services offered and Tiko
--> June recorded the highest calls

In [16]:
# Checking the variable types

cat_cols=call_center_clean.select_dtypes(include=['object']).columns
num_cols = call_center_clean.select_dtypes(include=np.number).columns.tolist()
print("Categorical Variables:")
print(cat_cols)
print("Numerical Variables:")
print(num_cols)

Categorical Variables:
Index(['Call status', 'Gender', 'Year of birth', 'Visited facility',
       'Were you given a tiko card?', 'Satisfaction', 'Feedback', 'Concerns',
       'Notes', 'Month'],
      dtype='object')
Numerical Variables:
[]


-> EDA Univariate Analysis

In [ ]:
fig, axes = plt.subplots(3, 2, figsize = (18, 18))
fig.suptitle('Bar plot for all categorical variables in the call_center_cleanset')
sns.countplot(ax = axes[0, 0], x = 'Call status', call_center_clean = call_center_clean, color = 'blue', 
              order = call_center_clean['Call status'].value_counts().index);
sns.countplot(ax = axes[0, 1], x = 'Gender', call_center_clean = call_center_clean, color = 'blue', 
              order = call_center_clean['Gender'].value_counts().index);
sns.countplot(ax = axes[1, 0], x = 'Year of birth', call_center_clean = call_center_clean, color = 'blue', 
              order = call_center_clean['Year of birth'].value_counts().index);
sns.countplot(ax = axes[1, 1], x = 'Visited facility', call_center_clean = call_center_clean, color = 'blue', 
              order = call_center_clean['Visited facility'].value_counts().index);
sns.countplot(ax = axes[2, 0], x = 'Were given a tiko card?', call_center_clean = call_center_clean, color = 'blue', 
              order = call_center_clean['Were given a tiko card?'].head(20).value_counts().index);
sns.countplot(ax = axes[2, 1], x = 'Satisfaction', call_center_clean = call_center_clean, color = 'blue', 
              order = call_center_clean['Satisfaction'].head(20).value_counts().index);
sns.countplot(ax = axes[2, 2], x = 'Feedback', call_center_clean = call_center_clean, color = 'blue', 
              order = call_center_clean['Feedback'].value_counts().index);
sns.countplot(ax = axes[3, 0], x = 'Concerns', call_center_clean = call_center_clean, color = 'blue', 
              order = call_center_clean['Concerns'].head(20).value_counts().index);
sns.countplot(ax = axes[3, 1], x = 'Month', call_center_clean = call_center_clean, color = 'blue', 
              order = call_center_clean['Month'].head(20).value_counts().index);
sns.countplot(ax = axes[3, 2], x = 'Notes', call_center_clean = call_center_clean, color = 'blue', 
              order = call_center_clean['Notes'].value_counts().index);
axes[1][1].tick_params(labelrotation=45);
axes[2][0].tick_params(labelrotation=90);
axes[2][1].tick_params(labelrotation=90);

In [ ]:
# Set the sefault Seaborn style for charts

sns.set()
plt.figure(figsize=(10, 6))
sns.heatmap(data=average_calls, cmap= 'Month')

plt.title('Average...')
plt.xlabel('Call status')
plt.ylabel('Rafiki Id')

plt.show()

In [ ]:
# Plotting the graph for all the years with promo available or not

sns.catplot(data = call_center_clean, x ="Month of Service Uptake", y = 'Visited facility',col = 'Gender',  hue = 'Satisfaction', row = 'Year of birth' ,sharex=False)
#plt.savefig('../charts/sales_vs_promotion.jpg')